[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Fuenfgeld/2022TeamBDataEngineeringBC/blob/main/Uebungen/Loesungen/Loesungen%20zu%20Teil%203.ipynb)

# Lösungen zu Teil 3

## Bitte führen Sie die nächsten beiden Codeabschnitte aus

In [ ]:
#Load Libraries
import sqlite3
import pandas as pd

# Load GitHub repo
%cd /content/
%rm -r ./2022TeamBDataEngineeringBC
!git clone https://github.com/Fuenfgeld/2022TeamBDataEngineeringBC.git
%cd ./2022TeamBDataEngineeringBC/

In [ ]:
# Connect to Database
conn = sqlite3.connect('sqlite_workshop.db')
cursor = conn.cursor()
print("Opened database successfully");

# Delete table if exists 
cursor.execute("DROP TABLE IF EXISTS food")
cursor.execute("DROP TABLE IF EXISTS orders")
cursor.execute("DROP TABLE IF EXISTS animals")
cursor.execute("DROP TABLE IF EXISTS customer")
cursor.execute("DROP TABLE IF EXISTS customer_buys_food")
cursor.execute("DROP TABLE IF EXISTS employee")
cursor.execute("DROP TABLE IF EXISTS sales")
cursor.execute("DROP TABLE IF EXISTS food_new")
cursor.execute("DROP TABLE IF EXISTS customer_new")
cursor.execute("DROP TABLE IF EXISTS employee_new")
cursor.execute("DROP TABLE IF EXISTS Liga")
cursor.execute("DROP TABLE IF EXISTS Verein")
cursor.execute("DROP TABLE IF EXISTS Spieler")


# Daten einlesen
food = pd.read_csv('tables/initial_tables/Futter.csv', sep=";")
customer = pd.read_csv('tables/initial_tables/Kunden.csv', sep=";")
animals = pd.read_csv('tables/initial_tables/Tiere.csv', sep=";")
orders = pd.read_csv('tables/initial_tables/Order.csv', sep=";")
employee = pd.read_csv('tables/initial_tables/Verkaeufer.csv', sep=";")
customer_buys_food = pd.read_csv('tables/initial_tables/Customer_buys_Food.csv', sep=";")


# Tabellen erstellen
cursor.execute('''
		CREATE TABLE IF NOT EXISTS customer (
			customer_ID int primary key,
            name nvarchar(50),
			postal_code nvarchar(5),
			city nvarchar(50)
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS employee (
			employee_ID int primary key,
            name nvarchar(50),
            gender nvarchar(1),
			wage int
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS food (
			food_ID int primary key,
            name nvarchar(50),
			price_per_kg float,
            producer nvarchar(50),
            origin nvarchar(50)
			)
               ''')

cursor.execute('''
		CREATE TABLE IF NOT EXISTS animals (
            name nvarchar(50) primary key,
			food int,
			foreign key(food) references food(food_ID)
			)
               ''') 
cursor.execute('''
		CREATE TABLE IF NOT EXISTS orders (
            order_Nr int primary key,
            date Date,
            transact int
			)
               ''')


cursor.execute('''
		CREATE TABLE IF NOT EXISTS customer_buys_food (
            C_ID int,
            F_ID int,
			amount int,
            O_ID int,
            E_ID int,
            foreign key(C_ID) references customer(customer_ID)
			foreign key(F_ID) references food(food_ID)
            foreign key(O_ID) references orders(order_Nr)
            foreign key(E_ID) references employee(employee_ID)
			)
               ''') 

# Insert Dataframe in Table
customer.to_sql('customer', conn, if_exists='append', index=False)
animals.to_sql('animals', conn, if_exists='append', index=False)
food.to_sql('food', conn, if_exists='append', index=False)
orders.to_sql('orders', conn, if_exists='append', index=False)
employee.to_sql('employee', conn, if_exists='append', index=False)
customer_buys_food.to_sql('customer_buys_food', conn, if_exists='append', index=False)



# Sternschema erstellen
#cursor.execute('''PRAGMA foreign_keys = OFF;''')  # disable foreign keys
#conn.commit()
cursor.execute('''CREATE TABLE IF NOT EXISTS food_new(
                  food_id INT PRIMARY KEY,
                  name TEXT,
                  price_per_kg FLOAT,
                  producer TEXT,
                  origin TEXT,
                  animal TEXT
                  );''')

cursor.execute('''CREATE TABLE IF NOT EXISTS customer_new(
                  cust_id INT PRIMARY KEY NOT NULL UNIQUE,
                  name TEXT,
                  postal_code nvarchar(5),
                  city TEXT
                  );''')

cursor.execute('''CREATE TABLE IF NOT EXISTS employee_new(
                  empl_id INT PRIMARY KEY NOT NULL UNIQUE,
                  name TEXT,
                  gender nvarchar(1),
                  wage INT
                  );''')
conn.commit()

# Tabellen mit Daten füllen
cursor.execute('''INSERT INTO food_new(food_id, name, price_per_kg, producer, origin, animal)
                  SELECT food.food_ID, food.name, food.price_per_kg, food.producer, food.origin, animals.name
                  FROM food
                  INNER JOIN animals ON food.food_ID = animals.food;
                  ''')
conn.commit()
    
cursor.execute('''INSERT INTO employee_new(empl_id, name, gender, wage)
                  SELECT employee_ID, name, gender, wage
                  FROM employee;
                  ''')    
    
cursor.execute('''INSERT INTO customer_new(cust_id, name, postal_code, city)
                  SELECT customer_ID, name, postal_code, city
                  FROM customer;
                  ''')

conn.commit()
cursor.execute('''
    CREATE TABLE IF NOT EXISTS sales(
        order_nr  INT PRIMARY KEY NOT NULL UNIQUE,
        transact INT,
        date DATE,
        custumer_id int, 
        food_id INT,
        amount INT,
        employee_id INT,
            foreign key(food_id) references food_new(food_id),
            foreign key(custumer_id) references customer_new(cust_id),
            foreign key(employee_id) references employee_new(empl_id)
        );''')

cursor.execute('''
    INSERT INTO sales( order_nr, transact, date, custumer_id, food_id, amount,employee_id )
    SELECT order_Nr,transact, date, C_ID, food_ID, amount, E_ID 
    FROM orders
    LEFT JOIN 
        customer_buys_food ON orders.order_Nr = customer_buys_food.O_ID
    LEFt JOIN 
        food ON customer_buys_food.F_ID = food.food_ID;
    ''')
cursor.execute(''' PRAGMA table_info(sales);''')
records = cursor.fetchall()

# Aufgabe zu Teil 3

### Hiermit werden die Inhalte der Dimensionstabellen angezeigt

In [ ]:
print("Tabelle customer_new")
print(pd.read_sql_query("SELECT * FROM customer_new", conn))
print("--------------------------------------------------------")
print("Tabelle food_new")
print(pd.read_sql_query("SELECT * FROM food_new", conn))
print("--------------------------------------------------------")
print("Tabelle employee_new")
print(pd.read_sql_query("SELECT * FROM employee", conn))
print("--------------------------------------------------------")

### Hiermit werden die Inhalte der Faktentabelle angezeigt

In [ ]:
print(pd.read_sql_query("SELECT * FROM sales", conn))

#### Beispiel: Welcher Verkäufer hat am meisten Umsatz erwirtschaftet?

In [ ]:
print(pd.read_sql_query('''
    SELECT 
    employee_new.name as employee, sales.amount, food_new.price_per_kg,
    SUM(food_new.price_per_kg*sales.amount) as sum_sales
    FROM employee_new
    LEFT JOIN 
        sales ON employee_new.empl_id = sales.employee_id
    INNER JOIN 
        food_new ON sales.food_id = food_new.food_id
    GROUP BY employee_new.name
    ORDER BY sum_sales DESC
    ''', conn))### Aufgabe 3.1: Welche Futter (name) wurde am meisten verkauft?

### Aufgabe 3.1: Welches Futter (name) wurde am meisten verkauft?

In [ ]:
print(pd.read_sql_query('''
    SELECT sales.food_id, food_new.name FROM sales
    LEFT JOIN
    food_new ON sales.food_id = food_new.food_id
    ''', conn))

### Aufgabe 3.2: Welcher Verkäufer hat am häufigsten Bestellungen abgeschloßen?

In [ ]:
print(pd.read_sql_query('''
    SELECT employee_id, COUNT(order_nr) as count_order FROM sales
    GROUP BY employee_id
    ''', conn))